In [386]:
# загружаем данные
from os import path, environ
import pandas as pd
import numpy as np

data_dir = "./lastfm-dataset-360K"
environ["DATA_DIR"] = "./lastfm-dataset-360K"

col_names = ["user", "artist-mbid", "artist-name", "T_plays"]
data = pd.read_csv(
    path.join(data_dir, "lastfm_small.tsv"),
    sep="\t",
    header=None,
    names=col_names
)

data.head()

,user,artist-mbid,artist-name,T_plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [387]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,T_plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [388]:
data.describe()

,T_plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,216.606950,10232.925996,33678.492236
std,604.378024,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,34.000000,5118.000000,17298.000000
50%,94.000000,10237.000000,34544.000000
75%,225.000000,15347.000000,49488.000000
max,135392.000000,20465.000000,66799.000000


In [389]:
data.T_plays = 1+log(data.T_plays) # проведем логарифмирование кол-ва прослушиваний чтобы уменьшить выбросы и прибавим 1 чтобы не получился 0 где у нас одно прослушивание

In [390]:
data.describe()

,T_plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,5.418532,10232.925996,33678.492236
std,1.472299,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,4.526361,5118.000000,17298.000000
50%,5.543295,10237.000000,34544.000000
75%,6.416100,15347.000000,49488.000000
max,12.815930,20465.000000,66799.000000


In [391]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [392]:
test_data.head()

,T_plays,user_id,artist_id
492245,4.401197,10077,16424
812744,4.931826,16641,43328
467986,5.499810,9574,15895
460551,6.834811,9420,36846
98775,3.302585,2007,50662


In [393]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 2
нет в тестовой выборке, но есть в обучающей: 23
всего пользователей: 20465


In [394]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

In [395]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "T_plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train_data[["user_id", "artist_id", "T_plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [396]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [397]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["T_plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799844
Доля ненулевых элементов: 0.0005850538734901935
Среднее значение ненулевых элементов: 5.419256210327148
Максимальное значение ненулевых элементов: 14.960376739501953
Минимальное значение ненулевых элементов: 1.0


In [398]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
sparse_info(model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 1.597001552581787 секунд
Размерности матрицы: (66800, 66800)
Ненулевых элементов в матрице: 1217509
Доля ненулевых элементов: 0.0002728470454300979
Среднее значение ненулевых элементов: 0.4217938860304889
Максимальное значение ненулевых элементов: 1.0000000577920343
Минимальное значение ненулевых элементов: 0.0


In [399]:
pre = []
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
for user_id in test_data["user_id"].unique():
    for artist_id, score in model.recommend(user_id, user_plays):
         pre.append((user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 32.1083402633667 секнуд


In [400]:
truth = test_data.sort_values(['user_id'])[['user_id','artist_id', 'T_plays']]

In [401]:
pred = pd.DataFrame(pre, columns=["user_id","artist_id","plays"])

In [402]:
# вычисление mrr predicted - DataFrame('user_id','artist_id', 'plays'), actual - DataFrame('user_id','artist_id', 'total-plays')

def mean_rr(predicted, actual):
    mrr = 0
    
    
    for x in pred.user_id.unique():
        mrr += rr(predicted.artist_id[predicted.user_id == x].tolist(),  actual.artist_id[actual.user_id == x].tolist())
    mrr = mrr/ len (pred.user_id.unique())
    print("mrr=: {}".format(mrr))
    return mrr


def rr(predicted, actual):
    for i, x in enumerate(predicted):
        if x in actual:
            return 1.0 / (i + 1)
    return 0

In [403]:
mean_rr(pred, truth)

mrr=: 0.3271012720156515


0.3271012720156515

### ALS

In [246]:
import implicit



In [429]:
# train the model on a sparse matrix of item/user/confidence weights
import time
start = time.time()
model = implicit.als.AlternatingLeastSquares(factors=100, iterations=100, regularization=0.1)
print("строим матрицу схожести по ALS")
model.fit(plays)
print("построили матрицу схожести по ALS за {} секунд".format(
        time.time() - start))


строим матрицу схожести по ALS
построили матрицу схожести по ALS за 92.80302309989929 секунд


In [430]:
pre = []
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
for user_id in test_data["user_id"].unique():
    for artist_id, score in model.recommend(user_id, user_plays):
         pre.append((user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 72.3310215473175 секнуд


In [431]:
truth = test_data.sort_values(['user_id'])[['user_id','artist_id', 'T_plays']]

In [432]:
pred = pd.DataFrame(pre, columns=["user_id","artist_id","plays"])

In [433]:
mean_rr(pred, truth)

mrr=: 0.3922793386761025


0.3922793386761025

## ALS

### Если логорифмировать количество прослушиваний
#### factors=100  mrr=: 0.38975258481884956  
#### factors=300  mrr=: 0.381210125316545
#### factors=50   mrr=: 0.38927901836660767
#### iterations=50 factors=100 mrr=: 0.39202213990618423
#### factors=100, iterations=50, regularization=0.05   mrr=: 0.39178687913520927
#### factors=100, iterations=50, regularization=0.005  mrr=: 0.39023281458701453
#### factors=100, iterations=100, regularization=0.001 mrr=: 0.3923675177833614
#### factors=100, iterations=100, regularization=0.01  mrr=: 0.39188903643648504
#### factors=100, iterations=100, regularization=0.1  mrr=: 0.3922793386761025


### и без логарифмирования
#### factors=50   mrr=: 0.29545562714096
#### actors=100   mrr=: 0.3158930473643789



## COS

### Если логорифмировать количество прослушиваний
#### mrr=: 0.3271012720156515
### и без логарифмирования
#### mrr=: 0.049374625847012604  совсем грусно :(
